<a href="https://colab.research.google.com/github/Shopping-Yuan/pytorch_practice/blob/readme-edit/ML2021HW1_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pytorch 設置:
torch.backends.cudnn 設定捲積的優化方式，無捲積層故關閉。

In [ ]:
#import pytorch and set CNN algorithm
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)